## 2. Classificação e Análise de Sentimento

Vamos classificar as notícias por categoria e analisar o sentimento de cada uma.

In [ ]:
# Classifica notícias
news_df = analyzer.classify_news(news_df)

# Analisa sentimento
news_df = analyzer.analyze_sentiment(news_df)

# Mostra distribuição por categoria
category_counts = pd.Series([
    cat for cats in news_df['categories'] for cat in cats
]).value_counts()

print('
Distribuição por categoria:')
print(category_counts)

print('
Distribuição de sentimento:')
print(news_df['sentiment'].value_counts())

# Visualiza distribuição
import plotly.express as px

fig = px.pie(values=category_counts.values,
            names=category_counts.index,
            title='Distribuição de Notícias por Categoria')
fig.show()

## 3. Cálculo de Impacto

Vamos calcular o score de impacto de cada notícia baseado em sua categoria e sentimento.

In [ ]:
# Calcula scores de impacto
news_df = analyzer.calculate_impact_score(news_df)

# Visualiza distribuição dos scores
fig = px.histogram(news_df, x='impact_score',
                  title='Distribuição dos Scores de Impacto',
                  nbins=30)
fig.show()

# Mostra média de impacto por categoria
impact_by_category = []
for cat in analyzer.event_categories.keys():
    mask = news_df['categories'].apply(lambda x: cat in x)
    avg_impact = news_df[mask]['impact_score'].mean()
    impact_by_category.append({
        'category': cat,
        'avg_impact': avg_impact
    })

impact_df = pd.DataFrame(impact_by_category)
print('
Impacto médio por categoria:')
print(impact_df.sort_values('avg_impact', ascending=False))

## 4. Correlação com Preços

Vamos analisar como os scores de impacto se correlacionam com os movimentos do WDO.

In [ ]:
# Coleta dados de preço
price_df = fetch_wdo_data(
    start_date.strftime('%Y-%m-%d'),
    end_date.strftime('%Y-%m-%d')
)

# Calcula correlações
combined_df, correlations = analyzer.correlate_with_price(
    news_df, price_df, window='1D'
)

print('
Correlações:')
for metric, value in correlations.items():
    print(f'{metric}: {value:.3f}')

# Visualiza relação entre impacto e retornos
fig = px.scatter(combined_df, x='sum', y='returns',
                title='Impacto vs Retornos',
                labels={'sum': 'Score de Impacto Acumulado',
                        'returns': 'Retorno Diário'})
fig.show()

## 5. Geração de Sinais

Por fim, vamos gerar sinais de trading baseados na análise fundamentalista.

In [ ]:
# Gera sinais
signals = analyzer.generate_signals(news_df, threshold=0.5)

# Combina sinais com preços
signals_df = pd.DataFrame({
    'price': price_df['close'],
    'signal': signals
})

# Visualiza sinais
fig = go.Figure()

# Adiciona preço
fig.add_trace(go.Scatter(
    x=signals_df.index,
    y=signals_df['price'],
    name='Preço',
    line=dict(color='blue')
))

# Adiciona sinais de compra
buy_signals = signals_df[signals_df['signal'] == 1]
fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals['price'],
    mode='markers',
    name='Compra',
    marker=dict(color='green', size=10)
))

# Adiciona sinais de venda
sell_signals = signals_df[signals_df['signal'] == -1]
fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals['price'],
    mode='markers',
    name='Venda',
    marker=dict(color='red', size=10)
))

fig.update_layout(
    title='Sinais Fundamentalistas',
    xaxis_title='Data',
    yaxis_title='Preço'
)

fig.show()

# Estatísticas dos sinais
print('
Estatísticas dos sinais:')
print(f'Total de sinais: {len(signals[signals != 0])}')
print(f'Sinais de compra: {len(signals[signals == 1])}')
print(f'Sinais de venda: {len(signals[signals == -1])}')
print(f'Dias neutros: {len(signals[signals == 0])}')


## 6. Próximos Passos

1. Refinar categorias e pesos das notícias
2. Implementar mais fontes de dados
3. Melhorar análise de sentimento para contexto financeiro
4. Desenvolver backtesting dos sinais fundamentalistas
5. Integrar com análise técnica